In [2]:
import tensorflow as tf
import tensorflow.keras.optimizers as optimizers

import sys
sys.path.append("..")

from models.classification_model import ClassificationDGP
from likelihoods import Softmax
from utils_dataset import load_tf_dataset
from utils_training import MCEM_sampler_classification, MCEM_Q_maximizer, MCEM, MCEM_windows

In [3]:
# dataset information loading
dataset_name = 'mnist'
_, _, train_full_size, test_full_size = load_tf_dataset(dataset_name, data_dir='./tensorflow_datasets/')
dataset_size = train_full_size + test_full_size
print(f"Dataset {dataset_name}, total size:{dataset_size}")

############################## Getting data info:dataset name: boston ##############################
D: 13, N: 455, Ns: 51
X_mean: [1.7378345e+00 1.1138461e+01 1.1224440e+01 7.2527476e-02 5.5657738e-01
 6.2923098e+00 6.9054504e+01 3.6594815e+00 4.3472528e+00 4.0854724e+02
 1.8481098e+01 3.5551254e+02 1.2646418e+01], Y_mean: [22.656263], Y_std: [9.32293]
######################################################################
Dataset boston, total size:506


In [4]:
# model setting
d_in = 28 * 28
d_out = 10
n_gp = 30
model = ClassificationDGP(d_in, d_out, n_hidden_layers=2, n_rf=100, n_gp=n_gp,
                      likelihood=Softmax(),
                      kernel_type_list=['RBF','RBF'], kernel_trainable=True,
                      random_fixed=True, input_cat=True)
print(f"{model.n_hidden_layers}-layer DGPs with Kernel type {model.kernel_type_list[0]}")

2-layer DGPs with Kernel type RBF


In [5]:
# EM sampler settings
batch_size= 200
lr_mcmc_0 = 0.01
beta_mcmc = 0.0
sampler_EM = MCEM_sampler_classification(model, dataset_name=dataset_name, batch_size=batch_size,
                                         lr_0=lr_mcmc_0, momentum_decay=beta_mcmc,
                                         precond_type='identity', K_batches=None, second_moment_centered=None,
                                         resample_in_cycle_head=False, start_sampling_epoch=2000, epochs_per_cycle=50)
# Maximizer setttings
lr_maximizer = 0.01
optimizer = optimizers.Adam(learning_rate=lr_maximizer)
maximizer = MCEM_Q_maximizer(model, dataset_size, optimizer)
# sampler settings after fixing hyper-params
lr_fixing_hyper_0 = 0.01
beta_fixing = 0.0
sampler_fixing_hyper = MCEM_sampler_classification(model, dataset_name=dataset_name, batch_size=batch_size,
                                                   lr_0=lr_fixing_hyper_0, momentum_decay=beta_fixing,
                                                   precond_type='identity', K_batches=None, second_moment_centered=None,
                                                   resample_in_cycle_head=False,
                                                   start_sampling_epoch=2000, epochs_per_cycle=50)

############################## Getting data info:dataset name: boston ##############################
D: 13, N: 455, Ns: 51
X_mean: [1.7378345e+00 1.1138461e+01 1.1224440e+01 7.2527476e-02 5.5657738e-01
 6.2923098e+00 6.9054504e+01 3.6594815e+00 4.3472528e+00 4.0854724e+02
 1.8481098e+01 3.5551254e+02 1.2646418e+01], Y_mean: [22.656263], Y_std: [9.32293]
######################################################################
Training size is 400 after remainder dropping. 
############################## Getting data info:dataset name: boston ##############################
D: 13, N: 455, Ns: 51
X_mean: [1.7378345e+00 1.1138461e+01 1.1224440e+01 7.2527476e-02 5.5657738e-01
 6.2923098e+00 6.9054504e+01 3.6594815e+00 4.3472528e+00 4.0854724e+02
 1.8481098e+01 3.5551254e+02 1.2646418e+01], Y_mean: [22.656263], Y_std: [9.32293]
######################################################################
Training size is 400 after remainder dropping. 


In [ ]:
# MCEM settings and training
total_EM_steps = 20000
ds_train_M, _, _, _ = load_tf_dataset(dataset_name, batch_size=100)
log_p, acc = MCEM(sampler_EM, maximizer, sampler_fixing_hyper, total_EM_steps, ds_train_M,
                  num_samples_EM=100, num_samples_fixing_hyper=200,
                  print_epoch_cycle_EM=200, print_epoch_cycle_fixing=100)

In [ ]:
log_p

In [ ]:
drop_mixing = 0
log_p_droped = log_p[drop_mixing:, :]
mse_droped = mse[drop_mixing:, :]

n_models_droped = tf.shape(mse_droped)[0]
predict_log_p = tf.reduce_logsumexp(log_p_droped, axis=0) - tf.math.log(tf.cast(n_models_droped, tf.float32))
predict_log_p = tf.reduce_mean(predict_log_p)
predict_acc = tf.reduce_mean(mse_droped)

print(f"Dataset: {dataset_name}")
print(f"Number of sampled models(after dropping {drop_mixing} samples): {n_models_droped}")
print(f"Test Log Likelihood of all sampled models: {predict_log_p}")
print(f"Test Root MSE of all sampled models: {predict_rmse}")

log_p_coll.append(predict_log_p)
rmse_coll.append(predict_rmse)
print('*' * 70)
print('*' * 70)
print(' ')
